# **EPC. PREPARACION DE LAS BASES FINALES**

## **1. Matriz para funcion item similarity**

### **1.1. Importar base**

In [12]:
import pyarrow as pa
import pyarrow.parquet as pq

import scipy as sp
import re

from sklearn.metrics.pairwise import cosine_similarity
import operator

from math import sqrt
from numpy import mean, var
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy.stats import skew


In [13]:
ruta_archivo_parquet = "bases/df_user_model.parquet"
df_user_model = pd.read_parquet(ruta_archivo_parquet)

# Imprimir el DataFrame importado
print(df_user_model.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51967 entries, 0 to 51966
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    51967 non-null  object
 1   eval       51967 non-null  int64 
 2   item_name  51967 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB
None


In [14]:
# Buscar valores "Sin nombre" en la columna 'item_name'
sin_nombre_count = df_user_model['item_name'].eq('Sin nombre').sum()

# Mostrar la cantidad de valores "Sin nombre"
print(f"Cantidad de valores 'Sin nombre' en la columna 'item_name': {sin_nombre_count}")


Cantidad de valores 'Sin nombre' en la columna 'item_name': 0


In [15]:
# Eliminar las filas con 'item_name' igual a "Sin nombre"
df_user_model = df_user_model.loc[df_user_model['item_name'] != 'Sin nombre']

# Mostrar el DataFrame resultante
print(df_user_model)


                 user_id  eval                         item_name
0      76561197970982479     3                     Killing Floor
1      76561197970982479     4                        Zeno Clash
2      76561197970982479     3                        Metro 2033
3                js41637     3         Barbie™ Dreamhouse Party™
4                js41637     3            Euro Truck Simulator 2
...                  ...   ...                               ...
51962  76561198312638244     3             Half-Life: Blue Shift
51963  76561198312638244     4                         Half-Life
51964  76561198312638244     3                        Black Mesa
51965        LydiaMorley     3     Counter-Strike Nexon: Zombies
51966        LydiaMorley     4  Counter-Strike: Global Offensive

[51967 rows x 3 columns]


In [16]:
df_user_model

,user_id,eval,item_name
0,76561197970982479,3,Killing Floor
1,76561197970982479,4,Zeno Clash
2,76561197970982479,3,Metro 2033
3,js41637,3,Barbie™ Dreamhouse Party™
4,js41637,3,Euro Truck Simulator 2
...,...,...,...
51962,76561198312638244,3,Half-Life: Blue Shift
51963,76561198312638244,4,Half-Life
51964,76561198312638244,3,Black Mesa
51965,LydiaMorley,3,Counter-Strike Nexon: Zombies


### **1.2. CREAR MATRIZ USUARIO-ITEM**

La matriz usuario-ítem es  una estructura de datos que refleja las evaluaciones otorgadas ("eval") por los usuarios ("user_id") a distintos ítems ("item_name"). Cada fila de esta matriz representa a un usuario, cada columna representa a un ítem, y las celdas contienen las valoraciones asignadas por los usuarios a esos ítems.

In [17]:

user_item_matrix = pd.pivot_table(df_user_model, values='eval', index='user_id', columns='item_name', fill_value=0)

# Visualizar la matriz usuario-ítem
user_item_matrix


item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ace--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ionex--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2SV-vuLB-Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Azsael-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zvanik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zynxgameth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### **1.3. normalizar los datos de la matriz y transposicion**


In [18]:
# Definición de la función de normalización
def normalize_row(row):
    mean_val = np.mean(row)
    min_val = np.min(row)
    max_val = np.max(row)
    normalized_row = (row - mean_val) / (max_val - min_val)
    return normalized_row

# Aplicación de la función de normalización a cada fila de la matriz usuario-ítem
us_it_norm = user_item_matrix.apply(normalize_row, axis=1)

us_it_norm

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301
--ace--,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301
--ionex--,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,...,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603,-0.000603
-2SV-vuLB-Kg,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,...,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904
-Azsael-,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,...,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958,-0.001958
zvanik,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,...,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904,-0.000904
zynxgameth,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,...,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301,-0.000301


In [19]:
us_it_norm_trans = us_it_norm.T
us_it_norm_trans

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-GM-Dragon,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zynxgameth,zyr0n1c,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
0RBITALIS,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
"10,000,000",-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
100% Orange Juice,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
1001 Spikes,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
the static speaks my name,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
theBlu,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301


### **1.4. Eliminar columnas sin valores**

In [20]:
# Eliminar columnas donde todos los valores son 0
us_it_norm_trans = us_it_norm_trans.loc[:, (us_it_norm_trans != 0).any()]

us_it_norm_trans



user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-GM-Dragon,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zynxgameth,zyr0n1c,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
0RBITALIS,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
"10,000,000",-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
100% Orange Juice,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
1001 Spikes,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
the static speaks my name,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
theBlu,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301


observamos que no cambio el tamaño de la matriz

### **1.5. Cambiar a matriz dispersa**

formatos que almacenan solo los valores distintos de ceros. Elegimos CSR, que es idonea para operaciones que se realizan sobre columnas.

In [21]:
df_usit_disp=sp.sparse.csr_matrix(us_it_norm_trans.values)
df_usit_disp


<3319x23064 sparse matrix of type '<class 'numpy.float64'>'
	with 76549416 stored elements in Compressed Sparse Row format>

### 1.6. **Creación de matriz de similitud item_similarity**

In [22]:
item_simi = cosine_similarity(df_usit_disp)

In [23]:
item_simi_df = pd.DataFrame(item_simi, index = us_it_norm_trans.index, columns = us_it_norm_trans.index)


Ya es posible observar un puntaje los juegos parecidos

In [24]:
item_simi_df

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,1.000000,0.021942,0.011078,-0.000682,0.020927,0.006752,0.010435,0.019619,0.019910,0.006572,...,0.012973,0.010176,0.012260,0.006811,0.005543,0.004451,0.216437,0.021507,-0.011369,0.000690
0RBITALIS,0.021942,1.000000,0.005958,-0.000182,0.011108,0.003709,0.005601,0.010445,0.010593,0.003549,...,0.006930,0.005447,0.006535,0.003691,0.003025,0.002476,-0.000484,0.011403,-0.005703,0.000494
"10,000,000",0.011078,0.005958,1.000000,-0.000823,0.005602,0.001364,0.002572,0.005143,0.005245,0.001545,...,0.003319,0.002563,0.003185,0.001557,0.001189,0.000789,-0.000945,0.005805,-0.004221,-0.000267
100% Orange Juice,-0.000682,-0.000182,-0.000823,1.000000,-0.000359,-0.001140,-0.000690,-0.000588,-0.000537,-0.000608,...,-0.000578,-0.000546,-0.000433,-0.000732,-0.000775,-0.001005,-0.001398,-0.000258,-0.002526,-0.001056
1001 Spikes,0.020927,0.011108,0.005602,-0.000359,1.000000,0.003408,0.005277,0.009928,0.010076,0.003322,...,0.006563,0.005147,0.006204,0.003442,0.002799,0.002244,-0.000639,0.010887,-0.005780,0.000340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.004451,0.002476,0.000789,-0.001005,0.002244,0.000079,0.000797,0.001946,0.002012,0.000393,...,0.001167,0.000857,0.001175,0.000343,0.000166,1.000000,-0.001026,0.002377,-0.002933,-0.000583
the static speaks my name,0.216437,-0.000484,-0.000945,-0.001398,-0.000639,-0.001187,-0.000808,-0.000840,-0.000795,-0.000675,...,-0.000737,-0.000667,-0.000588,-0.000798,-0.000821,-0.001026,1.000000,-0.000550,-0.002260,-0.001021
theBlu,0.021507,0.011403,0.005805,-0.000258,0.010887,0.003580,0.005462,0.010224,0.010371,0.003452,...,0.006773,0.005318,0.006393,0.003584,0.002928,0.002377,-0.000550,1.000000,-0.005736,0.000428


### **1.7. Exportar la base final **


In [25]:
pq.write_table(pa.Table.from_pandas(us_it_norm_trans), 'bases/us_it_norm_trans.parquet')
pq.write_table(pa.Table.from_pandas(item_simi_df), 'bases/item_simi_df.parquet')


c:\Users\Legion 5 3060\.conda\envs\SoyHenryProyectconda\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


probemos que se importan correctamente

In [26]:
#matriz de similitud 
ruta_archivo_item_simi = 'bases/item_simi_df.parquet'
item_simi_df = pq.read_table(ruta_archivo_item_simi).to_pandas()

#us_it_norm_trans
ruta_archivo_us_it_norm_trans = 'bases/us_it_norm_trans.parquet'
us_it_norm_trans_df = pq.read_table(ruta_archivo_us_it_norm_trans).to_pandas()

In [27]:
item_simi_df

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,123 Slaughter Me Street,140,...,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theBlu,theHunter,theHunter: Primal
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,1.000000,0.021942,0.011078,-0.000682,0.020927,0.006752,0.010435,0.019619,0.019910,0.006572,...,0.012973,0.010176,0.012260,0.006811,0.005543,0.004451,0.216437,0.021507,-0.011369,0.000690
0RBITALIS,0.021942,1.000000,0.005958,-0.000182,0.011108,0.003709,0.005601,0.010445,0.010593,0.003549,...,0.006930,0.005447,0.006535,0.003691,0.003025,0.002476,-0.000484,0.011403,-0.005703,0.000494
"10,000,000",0.011078,0.005958,1.000000,-0.000823,0.005602,0.001364,0.002572,0.005143,0.005245,0.001545,...,0.003319,0.002563,0.003185,0.001557,0.001189,0.000789,-0.000945,0.005805,-0.004221,-0.000267
100% Orange Juice,-0.000682,-0.000182,-0.000823,1.000000,-0.000359,-0.001140,-0.000690,-0.000588,-0.000537,-0.000608,...,-0.000578,-0.000546,-0.000433,-0.000732,-0.000775,-0.001005,-0.001398,-0.000258,-0.002526,-0.001056
1001 Spikes,0.020927,0.011108,0.005602,-0.000359,1.000000,0.003408,0.005277,0.009928,0.010076,0.003322,...,0.006563,0.005147,0.006204,0.003442,0.002799,0.002244,-0.000639,0.010887,-0.005780,0.000340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.004451,0.002476,0.000789,-0.001005,0.002244,0.000079,0.000797,0.001946,0.002012,0.000393,...,0.001167,0.000857,0.001175,0.000343,0.000166,1.000000,-0.001026,0.002377,-0.002933,-0.000583
the static speaks my name,0.216437,-0.000484,-0.000945,-0.001398,-0.000639,-0.001187,-0.000808,-0.000840,-0.000795,-0.000675,...,-0.000737,-0.000667,-0.000588,-0.000798,-0.000821,-0.001026,1.000000,-0.000550,-0.002260,-0.001021
theBlu,0.021507,0.011403,0.005805,-0.000258,0.010887,0.003580,0.005462,0.010224,0.010371,0.003452,...,0.006773,0.005318,0.006393,0.003584,0.002928,0.002377,-0.000550,1.000000,-0.005736,0.000428


In [28]:
us_it_norm_trans_df

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-GM-Dragon,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zynxgameth,zyr0n1c,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
0RBITALIS,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
"10,000,000",-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
100% Orange Juice,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
1001 Spikes,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
the static speaks my name,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
theBlu,-0.000301,-0.000301,-0.000603,-0.000904,-0.000301,-0.000301,-0.000402,-0.000301,-0.000603,-0.000402,...,-0.000829,-0.000301,-0.000301,-0.000301,-0.001055,-0.001958,-0.000904,-0.000301,-0.001883,-0.000301
